## Generate customer, category and products data.

In [1]:
import pandas as pd
import random
from faker import Faker

fake = Faker()

# Generate Customer Data
customers = []
for _ in range(150):
    customers.append({
        "Cust_Email": fake.email(),
        "Cust_Name": fake.name(),
        "Cust_Password": fake.password(),
        "Cust_Birthdate": fake.date_of_birth(minimum_age=18, maximum_age=80),
        "Cust_Phone": fake.phone_number(),
    })
df_customers = pd.DataFrame(customers)

# Define specific categories
category_data = [
    {"Category_ID": 1, "Category_Name": "Electronics", "Category_Description": "This category is related to items of Electronics."},
    {"Category_ID": 2, "Category_Name": "Clothing", "Category_Description": "This category is related to items of Clothing."},
    {"Category_ID": 3, "Category_Name": "Books", "Category_Description": "This category is related to items of Books."},
    {"Category_ID": 4, "Category_Name": "Kitchenware", "Category_Description": "This category is related to items of Kitchenware."},
    {"Category_ID": 5, "Category_Name": "Toys", "Category_Description": "This category is related to items of Toys."}
]
df_categories = pd.DataFrame(category_data)

# Generate Product Data
products = []
for i in range(1, 31):
    category = random.choice(category_data)
    products.append({
        "Prod_Number": i,
        "Prod_Name": f"{category['Category_Name']} Product {i}",
        "Prod_Brand": fake.company(),
        "Prod_Description": f"A {category['Category_Name']} item.",
        "Prod_Warranty": random.choice(["6 months", "1 year", "2 years"]),
        "Prod_Stock": random.randint(10, 100),
        "Prod_Dimensions": f"{random.randint(10, 100)}x{random.randint(10, 100)}x{random.randint(10, 100)} cm",
        "Prod_Weight": f"{random.uniform(0.5, 5.0):.2f} kg",
        "Prod_Colour": fake.color_name(),
        "Prod_Price": round(random.uniform(10, 1000), 2),
        "Category_ID": category['Category_ID']
    })
df_products = pd.DataFrame(products)


## Generate payment method data

In [23]:
# Generate Payment Method Data (must exist before being used in other tables)
payment_methods = []
for i in range(1, 151):
    payment_methods.append({
        "Payment_ID": i,
        "Cust_Email": df_customers["Cust_Email"][i % 150],
        "Payment_Type": random.choice(["CreditDebit", "GiftCard"])
    })
df_payment_methods = pd.DataFrame(payment_methods)

# Generate Credit/Debit Card Data
credit_debit_cards = []
gift_cards = []

# Iterate over all payment methods to assign based on type
for _, row in df_payment_methods.iterrows():
    if row["Payment_Type"] == "CreditDebit":
        credit_debit_cards.append({
            "Payment_ID": row["Payment_ID"],
            "Card_Number": fake.credit_card_number(),
            "Name_On_Card": df_customers[df_customers["Cust_Email"] == row["Cust_Email"]]["Cust_Name"].values[0],
            "Expiry_Date": fake.credit_card_expire(),
            "Verification_Code": random.randint(100, 999),
            "Is_Default_Payment": random.choice([True, False])
        })
    elif row["Payment_Type"] == "GiftCard":
        gift_cards.append({
            "Payment_ID": row["Payment_ID"],
            "Serial_Number": fake.bothify(text='GC########'),
            "Expiry_Date": fake.date_this_decade(),
            "Total_Amount_Loaded": round(random.uniform(10, 500), 2),
            "Current_Balance": round(random.uniform(0, 500), 2)
        })

df_credit_debit_cards = pd.DataFrame(credit_debit_cards)
df_gift_cards = pd.DataFrame(gift_cards)



## Generate order data

In [3]:
# Generate Order Data
orders = []
order_details = []
customer_pool = random.choices(df_customers["Cust_Email"], k=100)  # Use a subset of customers multiple times
for i in range(1, 151):
    cust_email = random.choice(customer_pool)
    payment_id = df_payment_methods[df_payment_methods["Cust_Email"] == cust_email].sample(n=1).iloc[0]["Payment_ID"]
    orders.append({
        "Order_Number": i,
        "Order_Date": fake.date_this_year(),
        "Order_Subtotal": 0,  # Placeholder
        "Order_GrandTotal": 0,  # Placeholder
        "Cust_Email": cust_email,
        "Payment_ID": payment_id
    })

    # Each order has 1 to 3 products
    num_products = random.randint(1, 3)
    products_in_order = random.sample(list(df_products["Prod_Number"]), num_products)
    for prod_number in products_in_order:
        quantity = random.randint(1, 5)
        price_at_purchase = df_products[df_products["Prod_Number"] == prod_number]["Prod_Price"].values[0]
        order_details.append({
            "Order_Number": i,
            "Prod_Number": prod_number,
            "Quantity": quantity,
            "Price_At_Purchase": price_at_purchase
        })

df_orders = pd.DataFrame(orders)
df_order_details = pd.DataFrame(order_details)

# Update Order Subtotals and Grand Totals
for i in df_orders["Order_Number"]:
    order_items = df_order_details[df_order_details["Order_Number"] == i]
    subtotal = (order_items["Quantity"] * order_items["Price_At_Purchase"]).sum()
    df_orders.loc[df_orders["Order_Number"] == i, "Order_Subtotal"] = subtotal
    df_orders.loc[df_orders["Order_Number"] == i, "Order_GrandTotal"] = subtotal + random.uniform(5, 15)  # Including some shipping/tax

## Generate basket data.

In [4]:
# Generate Basket and Basket Details Data
baskets = []
basket_details = []
for i in range(1, 151):
    cust_email = random.choice(df_customers["Cust_Email"])
    basket_id = i
    baskets.append({
        "Basket_ID": basket_id,
        "Cust_Email": cust_email
    })
    # Each basket has 1 to 2 products
    num_products = random.randint(1, 2)
    products_in_basket = random.sample(list(df_products["Prod_Number"]), num_products)
    for prod_number in products_in_basket:
        quantity = random.randint(1, 3)
        basket_details.append({
            "Basket_ID": basket_id,
            "Prod_Number": prod_number,
            "Quantity": quantity
        })

df_baskets = pd.DataFrame(baskets)
df_basket_details = pd.DataFrame(basket_details)


## Generate customer and delivery address, and delivery data.

In [30]:
# Generate Customer Address Data
customer_addresses = []
for i in range(1, 151):
    customer_addresses.append({
        "Addr_ID" : i,
        "Cust_Email": df_customers["Cust_Email"][i % 150],
        "Addr_BuildingName": fake.building_number(),
        "Addr_BuildingNumber": fake.building_number(),
        "Addr_Street": fake.street_name(),
        "Addr_City": fake.city(),
        "Addr_Country": fake.country(),
        "Addr_Postcode": fake.postcode()
    })
df_customer_addresses = pd.DataFrame(customer_addresses)

# Generate Delivery Address Data
# Reuse customer address data for delivery, with some random different addresses
delivery_addresses = []
for i in range(1, 151):
    if random.choice([True, False]):  # 50% chance to use the customer's address
        customer_address = df_customer_addresses.iloc[i % 150]
        delivery_addresses.append({
            "Delivery_Addr_ID": i,
            "Delivery_Addr_BuildingName": customer_address["Addr_BuildingName"],
            "Delivery_Addr_BuildingNumber": customer_address["Addr_BuildingNumber"],
            "Delivery_Addr_Street": customer_address["Addr_Street"],
            "Delivery_Addr_City": customer_address["Addr_City"],
            "Delivery_Addr_Country": customer_address["Addr_Country"],
            "Delivery_Addr_Postcode": customer_address["Addr_Postcode"]
        })
    else:  # 50% chance to use a different random address
        delivery_addresses.append({
            "Delivery_Addr_ID": i,
            "Delivery_Addr_BuildingName": fake.building_number(),
            "Delivery_Addr_BuildingNumber": fake.building_number(),
            "Delivery_Addr_Street": fake.street_name(),
            "Delivery_Addr_City": fake.city(),
            "Delivery_Addr_Country": fake.country(),
            "Delivery_Addr_Postcode": fake.postcode()
        })

df_delivery_addresses = pd.DataFrame(delivery_addresses)

# Generate Delivery Data with verification
deliveries = []
for i in range(1, 151):
    # Ensure valid Order_Number and Delivery_Addr_ID
    order_number = df_orders["Order_Number"].sample(1).values[0]
    delivery_addr_id = df_delivery_addresses["Delivery_Addr_ID"].sample(1).values[0]

    # Verify the order number and address are valid
    if (order_number in df_orders["Order_Number"].values) and (delivery_addr_id in df_delivery_addresses["Delivery_Addr_ID"].values):
        delivery_date = pd.to_datetime(df_orders[df_orders["Order_Number"] == order_number]["Order_Date"].values[0]) + pd.DateOffset(days=5)
        deliveries.append({
            "Delivery_Track_Number": f"TRACK{i}",  # Unique tracking number
            "Order_Number": order_number,
            "Delivery_Addr_ID": delivery_addr_id,
            "Delivery_Date": delivery_date,
            "Delivery_Status": random.choice(["Pending", "Shipped", "Delivered", "Cancelled"])
        })

df_deliveries = pd.DataFrame(deliveries)



## Generate return data.

In [41]:
# Randomly select 10% of orders for returns
returned_orders = random.sample(list(df_orders["Order_Number"]), k=int(0.1 * len(df_orders)))

returns = []
for i, order_number in enumerate(returned_orders, start=1):
    # Get the delivery date for this order
    delivery_date = df_deliveries[df_deliveries["Order_Number"] == order_number]["Delivery_Date"].values[0]
    return_start_date = pd.to_datetime(delivery_date) + pd.DateOffset(days=3)
    return_due_date = return_start_date + pd.DateOffset(days=10)

    # Calculate Return Amount
    # Get the products associated with the returned order
    order_items = df_order_details[df_order_details["Order_Number"] == order_number]
    return_amount = (order_items["Quantity"] * order_items["Price_At_Purchase"]).sum()

    # Append the return details
    returns.append({
        "Return_ID": i,
        "Order_Number": order_number,
        "Return_Start_Date": return_start_date,
        "Return_Due_Date": return_due_date,
        "Return_Status": random.choice(["Pending", "Processed", "Rejected"]),
        "Return_Amount": return_amount
    })

df_returns = pd.DataFrame(returns)

# Generate return details records
return_details = []
for i, order_number in enumerate(returned_orders, start=1):
    order_items = df_order_details[df_order_details["Order_Number"] == order_number]
    for _, item in order_items.iterrows():
        return_details.append({
            "Return_ID": i,
            "Prod_Number": item["Prod_Number"],
            "Return_Reason": random.choice(["Defective product", "Wrong item delivered", "Product damaged", "Changed mind"]),
            "Quantity_Returned": random.randint(1, item["Quantity"])
        })

# Create DataFrame
df_return_details = pd.DataFrame(return_details)


In [ ]:
# Generate Review Data for all customers
reviews = []
review_qualifiers = ["good", "bad", "decent"]  # Possible review texts

for cust_email in df_customers["Cust_Email"]:
    for _ in range(2):  # Each customer writes 2 reviews
        prod_row = df_products.sample(1).iloc[0]
        prod_number = prod_row["Prod_Number"]
        prod_name = prod_row["Prod_Name"]
        
        review_text = f"{prod_name} is {random.choice(review_qualifiers)}"
        
        reviews.append({
            "Review_Number": f"RVW{len(reviews) + 1}",  # Unique Review Number
            "Review_Date": fake.date_this_year(),  # Random date within this year
            "Review_Text": review_text,  # Product-specific review text
            "Review_Ranking": random.randint(1, 5),  # Review ranking between 1 and 5
            "Cust_Email": cust_email,  # Foreign Key from Customer table
            "Prod_Number": prod_number  # Foreign Key from Product table
        })

# Create DataFrame
df_reviews = pd.DataFrame(reviews)



In [16]:
# Define the desired file path
file_path = r"C:\Users\user\OneDrive\Desktop\Second Year\ST207"

df_customers.to_csv(f"{file_path}\\Customer.csv", index=False)
df_categories.to_csv(f"{file_path}\\Category.csv", index=False)
df_products.to_csv(f"{file_path}\\Product.csv", index=False)
df_payment_methods.to_csv(f"{file_path}\\PaymentMethod.csv", index=False)
df_credit_debit_cards.to_csv(f"{file_path}\\CreditDebitCard.csv", index=False)
df_gift_cards.to_csv(f"{file_path}\\GiftCard.csv", index=False)
df_orders.to_csv(f"{file_path}\\Order.csv", index=False)
df_order_details.to_csv(f"{file_path}\\OrderDetails.csv", index=False)
df_baskets.to_csv(f"{file_path}\\Baskets.csv", index=False)
df_basket_details.to_csv(f"{file_path}\\BasketDetails.csv", index=False)
df_deliveries.to_csv(f"{file_path}\\Deliveries.csv", index=False)
df_customer_addresses.to_csv(f"{file_path}\\CustomerAddress.csv", index=False)
df_delivery_addresses.to_csv(f"{file_path}\\DeliveryAddress.csv", index=False)
df_returns.to_csv(f"{file_path}\\Returns.csv", index=False)
df_return_details.to_csv(f"{file_path}\\ReturnDetails.csv", index=False)
df_reviews.to_csv(f"{file_path}\\Reviews.csv", index=False)

In [ ]:
invalid_payments = df_payment_methods[~df_payment_methods["Cust_Email"].isin(df_customers["Cust_Email"])]

if not invalid_payments.empty:
    print("Mismatched payment methods found:")
    print(invalid_payments)
else:
    print("All payment methods are linked to valid customers.")